In [1]:
%load_ext autoreload
%autoreload 2

%cd "C:\Users\Derrick\Documents\DSC180B-PROJECT"

C:\Users\Derrick\Documents\DSC180B-PROJECT


In [2]:
import sys
import json
import subprocess
import os
import gzip
import pandas as pd
import tarfile
import numpy as np
import requests

# add library code to path
sys.path.insert(0, r'src/')

In [3]:
cfg = json.load(open("config/config.json"))

In [4]:
def mkdir(d):
    if not os.path.exists(d):
        os.mkdir(d)

In [5]:
def download_url(url, dirname, filename):
    mkdir(dirname)

    r = requests.get(url, stream=True)

    with open(dirname + filename, 'wb') as f:
        for chunk in r.iter_content():
            f.write(chunk)
            
file_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE44639&format=file'            
download_url(file_url, 'data/raw/', 'GSE44639_RAW.tar')

PermissionError: [Errno 13] Permission denied: 'data/raw/GSE44639_RAW.tar'

In [14]:

def read_file_type(health_types, cell_types, tmp='data/temp/'):
    tar = tarfile.open("data/raw/GSE44639_RAW.tar")

    mkdir(tmp)
    tar.extractall(tmp)
    
    df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)', 'healthy', 'subset', 'sample'])
    
    for name in tar.getnames():
        subset = name.split('_')[-1].split('.')[0]
        health_type = name[4:].split('_')[1][0]
        
        if subset in cell_types and health_type in health_types:
            df1 = pd.read_csv(gzip.open(tmp + name,'rt'), sep='\t')
            df1['healthy'] = 'M' in name[4:]
            df1['subset'] = subset
            df1['sample'] = name.split(".")[0]
            
            if len(df1.columns) != len(df.columns):
                print(df1)
                print(name)
                continue
            
            df = pd.concat([df, df1])
            
    return df

In [15]:
df = read_file_type(['P'], ['rTreg', 'aTreg', 'Naive', 'Tem', 'Tcm', 'Ttm'])
# df = read_file_type(['M'], ['rTreg', 'aTreg'])
df.to_csv("data/temp/all-PreType1.csv")

                       Sequence  Length      Chromosome   Position Strand  \
0           TAAGCCATGCATGTCTAAG      19  chrUn_gl000220     109121      +   
1           TAAGCCATGCATGTCTAAG      19  chrUn_gl000220     153093      +   
2           TAAGCCATGCATGTCTAAG      19           chr21    9827048      +   
3           TAAGCCATGCATGTCTAAG      19  chrUn_gl000229      18214      +   
4      ATCCCGGGGCCGAGGGAGCGAGAC      24  chrUn_gl000220     158263      +   
...                         ...     ...             ...        ...    ...   
44879     ATGGTTGCAAAGCTGAAACTT      21  chrUn_gl000220     154222      +   
44880     ATGGTTGCAAAGCTGAAACTT      21  chrUn_gl000220     110250      +   
44881     ATGGTTGCAAAGCTGAAACTT      21            chr2  133012584      -   
44882     ATGGTTGCAAAGCTGAAACTT      21  chrUn_gl000229      19349      +   
44883     ATGGTTGCAAAGCTGAAACTT      21            chrY   10036026      +   

             Gene      Region Proximity  Total reads  healthy subset  \
0  

In [229]:
df = read_file_type(['M'], ['rTreg', 'aTreg'])

healthy_mirnas = ['hsa-let-7a', 'hsa-miR-22*', 'hsa-mir-21', 'hsa-miR-221', 'hsa-miR-21*', 'hsa-miR-23b', 'hsa-miR-23a', 'hsa-miR-107', 'hsa-miR-21', 'hsa-mir-424', 'hsa-mir-532', 'hsa-miR-181a-2*', 'hsa-miR-151-3p', 'hsa-miR-193b', 'hsa-mir-30b', 'hsa-mir-3653', 'hsa-miR-335', 'hsa-mir-151', 'hsa-miR-151-5p', 'hsa-mir-31', 'hsa-miR-31', 'hsa-let-7c', 'hsa-mir-150', 'hsa-mir-26a-1', 'hsa-mir-26a-2']

healthy_a_treg_samples = df[df['subset'] == 'aTreg']
healthy_r_treg_samples = df[df['subset'] == 'rTreg']

a = healthy_a_treg_samples[healthy_a_treg_samples['miRNA'].apply(lambda x: x in healthy_mirnas)][['miRNA', 'Total miRNA reads', 'sample']].groupby(['miRNA', 'sample']).sum()
r = healthy_r_treg_samples[healthy_r_treg_samples['miRNA'].apply(lambda x: x in healthy_mirnas)][['miRNA', 'Total miRNA reads', 'sample']].groupby(['miRNA', 'sample']).sum()

a['Total miRNA reads'] = a['Total miRNA reads'].astype(float)
r['Total miRNA reads'] = r['Total miRNA reads'].astype(float)

a = a['Total miRNA reads'].reset_index()
a['name'] = a['miRNA'] + '_' + a['sample'].str.split('_',expand=True)[1]
a['Total miRNA reads A'] = a['Total miRNA reads']
a = a[['name', 'Total miRNA reads A']]
a['name'] = a['name'].astype(str)
a.set_index("name", inplace=True)

# a

r = r['Total miRNA reads'].reset_index()
r['name'] = r['miRNA'] + '_' + r['sample'].str.split('_',expand=True)[1]
r['Total miRNA reads R'] = r['Total miRNA reads']
r = r[['name', 'Total miRNA reads R']]
r['name'] = r['name'].astype(str)
r.set_index("name", inplace=True)

# r

d = a.join(r, on='name').fillna(0).reset_index()
d['sample'] = d['name'].str.split('_', expand=True)[1]
d['name'] = d['name'].str.split('_', expand=True)[0]
d['R/A'] = d['Total miRNA reads R'] / d['Total miRNA reads A']

f = d.groupby(['name', 'sample'])['R/A'].first().unstack().fillna(0).reset_index()
f['mean'] = f[f.columns[2:]].mean(axis=1)
f['std'] = f[f.columns[2:]].std(axis=1)

g = f[:]

# g[g.columns[2:]] / g['std'].values
# = (g[g.columns[2:]] - g['mean']) / g['std']
# g

for i in range(2, 10):
#     print(g.columns[i])
    g[g.columns[i]] = ((g[g.columns[i]] - g['mean']) / g['std']).fillna(0)
    
g = g.set_index('name')[g.columns[1:-2]]
g

sample,M10,M12,M26,M27,M29,M30,M7,M8,M9
name,,,,,,,,,
hsa-let-7a,4.700056,-0.355732,-0.100115,1.330123,-0.774222,2.004459,-0.720970,-0.687592,-0.695950
hsa-let-7c,7.071429,-0.760884,-0.677063,0.049747,-0.923615,1.421955,-0.155974,-0.802590,1.848423
hsa-miR-107,0.178049,2.364134,0.503844,-0.226474,-0.727694,0.225150,-0.695932,-0.719683,-0.723345
hsa-miR-151-3p,20.000000,2.645751,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964
hsa-miR-151-5p,31.000000,1.635000,1.680823,0.194112,-0.701987,-0.701987,-0.701987,-0.701987,-0.701987
hsa-miR-181a-2*,0.000000,2.645751,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964
hsa-miR-193b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hsa-miR-21,1.094187,2.272459,0.627335,-0.131183,-0.796824,0.289575,-0.750064,-0.725768,-0.785529
hsa-miR-21*,0.075758,2.214631,0.427154,0.634631,-0.790044,-0.116238,-0.790044,-0.790044,-0.790044


sample,name,M10,M12,M26,M27,M29,M30,M7,M8,M9,mean,std
0,hsa-let-7a,4.700056,-0.355732,-0.100115,1.330123,-0.774222,2.004459,-0.720970,-0.687592,-0.695950,1.558527,2.013023
1,hsa-let-7c,7.071429,-0.760884,-0.677063,0.049747,-0.923615,1.421955,-0.155974,-0.802590,1.848423,4.331468,4.689691
2,hsa-miR-107,0.178049,2.364134,0.503844,-0.226474,-0.727694,0.225150,-0.695932,-0.719683,-0.723345,0.371533,0.510563
3,hsa-miR-151-3p,20.000000,2.645751,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,0.500000,1.322876
4,hsa-miR-151-5p,31.000000,1.635000,1.680823,0.194112,-0.701987,-0.701987,-0.701987,-0.701987,-0.701987,15.319444,21.822972
5,hsa-miR-181a-2*,0.000000,2.645751,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,-0.377964,0.375000,0.992157
6,hsa-miR-193b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,hsa-miR-21,1.094187,2.272459,0.627335,-0.131183,-0.796824,0.289575,-0.750064,-0.725768,-0.785529,0.151540,0.190180
8,hsa-miR-21*,0.075758,2.214631,0.427154,0.634631,-0.790044,-0.116238,-0.790044,-0.790044,-0.790044,0.083751,0.106008
9,hsa-miR-22*,0.000000,2.640956,-0.261741,-0.290860,-0.417671,-0.417671,-0.417671,-0.417671,-0.417671,0.178572,0.427542


In [125]:
a[a.index == 'hsa-let-7a_GSM1088214_M7']

,Total miRNA reads
name,
hsa-let-7a_GSM1088214_M7,653.0


In [135]:
a

,Total miRNA reads
name,
hsa-let-7a_M7,653.0
hsa-let-7a_M8,3108.0
hsa-let-7a_M9,476.0
hsa-let-7a_10,1797.0
hsa-let-7a_12,10186.0
...,...
hsa-mir-31_26,4.0
hsa-mir-31_27,9.0
hsa-mir-31_29,6.0


In [136]:
r

,Total miRNA reads R
name,
hsa-let-7a_M7,70.0
hsa-let-7a_M8,542.0
hsa-let-7a_M9,75.0
hsa-let-7a_10,8446.0
hsa-let-7a_12,8581.0
...,...
hsa-mir-424_27,69.0
hsa-mir-532_M8,1.0
hsa-mir-532_10,3.0


In [31]:
# tar = tarfile.open("data/raw/GSE44639_RAW.tar")

# mkdir('data/temp')

# tar.extractall("data/temp")

# rTreg_M_df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)'])
# aTreg_M_df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)'])

# rTreg_P_df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)'])
# aTreg_P_df = pd.DataFrame(columns=['miRNA', 'Chromosome', 'Position', 'Strand', 'Total miRNA reads', 'RPM (reads per million)'])

# rTreg_M_dfs = []
# aTreg_M_dfs = []
# rTreg_P_dfs = []
# aTreg_P_dfs = []

# for name in tar.getnames():
    
    

#     if 'M' in name[4:]:
#         if 'rTreg' in name:
#             df = pd.read_csv(gzip.open('data/temp/' + name,'rt'), sep='\t')
#             if len(df.columns) == len(rTreg_M_df.columns) and (df.columns == rTreg_M_df.columns).all():
#                 rTreg_M_df = pd.concat([rTreg_M_df, df])
#                 rTreg_M_dfs.append(df)
#         elif 'aTreg' in name:
#             df = pd.read_csv(gzip.open('data/temp/' + name,'rt'), sep='\t')
#             if len(df.columns) == len(aTreg_M_df.columns) and (df.columns == aTreg_M_df.columns).all():
#                 aTreg_M_df = pd.concat([aTreg_M_df, df])
#                 aTreg_M_dfs.append(df)
#     elif 'P' in name[4:]:
#         if 'rTreg' in name:
#             df = pd.read_csv(gzip.open('data/temp/' + name,'rt'), sep='\t')
#             if len(df.columns) == len(rTreg_P_df.columns) and (df.columns == rTreg_P_df.columns).all():
#                 rTreg_P_df = pd.concat([rTreg_P_df, df])
#                 rTreg_P_dfs.append(df)
#         elif 'aTreg' in name:
#             df = pd.read_csv(gzip.open('data/temp/' + name,'rt'), sep='\t')
#             if len(df.columns) == len(aTreg_P_df.columns) and (df.columns == aTreg_P_df.columns).all():
#                 aTreg_P_df = pd.concat([aTreg_P_df, df])
#                 aTreg_P_dfs.append(df)

# tar.close()

# rTreg_M_df['healthy'] = True
# aTreg_M_df['healthy'] = True
# rTreg_P_df['healthy'] = False
# aTreg_P_df['healthy'] = False

# rTreg_M_df['subset'] = 'rTreg'
# rTreg_P_df['subset'] = 'rTreg'
# aTreg_M_df['subset'] = 'aTreg'
# aTreg_P_df['subset'] = 'aTreg'

# total_healthy = pd.concat([rTreg_M_df, aTreg_M_df])
# total_pt1 = pd.concat([rTreg_P_df, aTreg_P_df])

# total_both = pd.concat([total_healthy, total_pt1])
# total_both

In [81]:
import math

def absF(df, miRNA_name):
    # get only rows of the miRNA
    miRNAdf = df[df['miRNA'] == miRNA_name]
    miRNAdf['RPM (reads per million)'] = miRNAdf['RPM (reads per million)'].apply(lambda x: np.log2(x))
    N = len(miRNAdf)
    
    # calculate numerator
    meanY = miRNAdf['RPM (reads per million)'].astype(int).mean()
    groups = miRNAdf[['healthy', 'subset']].drop_duplicates()
    K = len(groups)
    
    numerator = 0
    denominator = 0
    
    ##
    
    if N - K == 0:
#         print(N, K, miRNA_name)
        return 0
    
    ##

    for row in groups.iterrows():
        health = row[1][0]
        cellType = row[1][1]

        groupDF = miRNAdf[(miRNAdf['healthy'] == health) & (miRNAdf['subset'] == cellType)]
        
        # mean Yi
        meanYi = groupDF['RPM (reads per million)'].astype(int).mean()
        numerator += np.abs(meanYi - meanY)*len(groupDF)/(K - 1)

        for sample in groupDF.iterrows():
#             if N - K == 0:
#                 print(N, K)
            
            denominator += np.abs(sample[1][5] - meanYi)/(N - K)
           
        
    if denominator == 0:
        return 0
    return numerator / denominator

In [84]:
df = read_file_type(['M'], ['rTreg', 'aTreg', 'Naive', 'Tem', 'Tcm', 'Ttm'])
absF(df, 'hsa-mir-26a-2')

# (total_both['miRNA'] == 'hsa-let-7a').sum()

r = []
w = []

for m in df['miRNA'].unique():
    a = absF(df, m)
    
    if a >= 9.6:
        r.append((m, a))
    else:
        w.append((m, a))

<ipython-input-81-1468f2390ee0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miRNAdf['RPM (reads per million)'] = miRNAdf['RPM (reads per million)'].apply(lambda x: np.log2(x))
<ipython-input-81-1468f2390ee0>:33: RuntimeWarning: invalid value encountered in double_scalars
  numerator += np.abs(meanYi - meanY)*len(groupDF)/(K - 1)


In [85]:
import pprint

pprint.pprint(r)


[('hsa-mir-124-2', 9.940107902417253), ('hsa-mir-345', 40.35393731905183)]


In [86]:
pprint.pprint(w)

[('hsa-miR-378c', 1.7429424142236887),
 ('hsa-mir-1321', 1.546405746949244),
 ('hsa-miR-378d', 2.4444444444444446),
 ('hsa-miR-2110', 4.197088481059411),
 ('hsa-mir-200b', 2.8736842105263154),
 ('hsa-miR-125a-5p', 4.613690242009389),
 ('hsa-mir-671', 3.4666666666666672),
 ('hsa-miR-193a-5p', 3.7333333333333334),
 ('hsa-mir-378', 4.067376382649623),
 ('hsa-mir-3202-1', 1.2345679012345678),
 ('hsa-mir-5096', 3.369836482359453),
 ('hsa-mir-320a', 3.490316832095838),
 ('hsa-miR-32', 2.213563098711687),
 ('hsa-miR-30b', 3.567950877335611),
 ('hsa-miR-30c', 2.226380788633123),
 ('hsa-miR-30d', 3.301293017933282),
 ('hsa-miR-30e', 2.7121043105482117),
 ('hsa-miR-30b*', 0),
 ('hsa-miR-643', 0),
 ('hsa-miR-1260', 3.2077813178222585),
 ('hsa-miR-17*', 2.9474065736344195),
 ('hsa-miR-150', 3.003191026775862),
 ('hsa-mir-101-2', 4.189938830630348),
 ('hsa-miR-331-5p', 0),
 ('hsa-mir-101-1', 3.3750713187903885),
 ('hsa-mir-27b', 0.8242962639568291),
 ('hsa-mir-128-1', 0.7778456398436235),
 ('hsa-mi

In [ ]:
# I am attempting to reproduce the absF function defined in your article.

# My first question is asking if the unit of Y(i,j) is log2 of 
# RPM of that specific miRNA

# For 3a, to calculate the absF value was the entire dataset used
# or was only the healthy set of aTreg and rTreg used? I am 
# also wondering if we are able to calculate these numbers using only the 
# dataset from 
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE44639

# In addition, what kind of cutoffs for expression level
# were defined to filter out miRNAs with large absF values?

# "... a cutoff was applied on expression level and on the absolute mean log2cFC
# to filter out those miRNAs with large absF values but low expression
# and small log2FC values"

In [210]:
# absF(total_both, 'hsa-let-7a')

df = pd.DataFrame(columns=['miRNA', 'absF'])

for miRNA in total_both['miRNA'].unique():
    d = pd.DataFrame([[miRNA, absF(total_both, miRNA)]], columns=['miRNA', 'absF'])
    df = pd.concat([df, d])
    
df

<ipython-input-209-b8ebed65b836>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miRNAdf['RPM (reads per million)'] = miRNAdf['RPM (reads per million)'].apply(lambda x: np.log2(x))
<ipython-input-209-b8ebed65b836>:32: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator


0.20863693391765029
0.34035079091222364
0.44956609018675253
0.5887014737121871
0.6432783371789002
0.7077671202141487
0.8923217488238412
1.0800313301141078
1.1385915868311491
1.194440582037411
1.2299446052769034
1.2443889787898923
1.284969402845848
1.2908451821823221
1.3791523534907604
1.389274662945546
1.4672932471989286
1.4906521054200674
1.5203129250163796
1.5440997345862033
1.5778414320294032
1.6415328882789009
1.6912375664635948
1.6934680173389478
1.738289935865678
1.8052284709412383
1.9123571824113468
1.9180139492693768
1.9244173601787595
1.9297787072080548
0.11363636363636363
0.22727272727272727
0.3832153335255858
0.4325318108310929
0.4698921787805184
0.5559066070886439
0.6403221915042283
0.7851972488776645
0.7975690832604692
0.8040625897539756
0.8430236287150146
0.9215608852413009
0.9605219242023398
1.0384440021244177
1.0463402445300092
1.0745755313433833
1.1183124962401696
1.1767540546817281
1.280650158577832
1.3878384269442157
1.4706533403481867
1.5388351585300049
1.6601953858

1.7465120722758412
1.8068641529506855
1.8439328525978749
1.8722656805714968
1.8738697612643271
1.9287329434845124
1.9739045322101032
2.097946354515308
2.1156802980928764
2.116118961110029
0.08771929824561404
0.09201951750590408
0.10956337715502688
0.1440508207455898
0.14835104000587984
0.2801014786782822
0.30265786965572583
0.33273305762565064
0.35528944860309425
0.37784583958053786
0.40792102755046267
0.4687837155583431
0.4913401065357867
0.5018664223252604
0.5123927381147341
0.5544980012726288
0.5650243170621025
0.5755506328515763
0.58607694864105
0.5966032644305237
0.6694960276263741
0.6800223434158478
0.6905486592053215
0.125
0.29000000000000004
0.41500000000000004
0.6168971003458193
0.7054911004612043
0.7456624211390029
0.7935395449344974
0.8776922925543327
0.8776922925543327
1.0076093930920762
1.0756269818177198
1.1036445705433635
1.190441570601056
1.2704415706010561
1.2781473737187519
1.3981473737187518
1.4373902639179763
1.4678763571042672
1.4935506809418497
1.527293430191077
1

<ipython-input-209-b8ebed65b836>:28: RuntimeWarning: invalid value encountered in double_scalars
  denominator += np.abs(sample[1][5] - meanYi)/(N - K)



0.09523809523809523
0.19047619047619047
0.23809523809523808
0.3075919818398713
0.4109217438133147
0.46449317238474325
0.4704455533371242
0.4763979342895051
0.5299693628609337
0.5869659388079509
0.6670780779535511
0.7206495065249797
0.8377905779878919
0.856838197035511
0.8854096256069396
0.9520762922736062
0.9806477208450348
1.0092191494164633
1.025092165289479
1.040965181162495
1.0727112129085268
1.0885842287815426
1.1203302605275745
1.1362032764005903
0.013888888888888888
0.027777777777777776
0.09722222222222222
0.1111111111111111
0.125
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.1388888888888889
0.0
0.0
0.0
0.1904761904761905
0.2857142857142857
0.38095238095238093
0.6370616598053281
0.6846807074243757
0.9227759455196138
0.9227759455196138
0.9227759455196138
0.18473353481202454
0.26229028647440267
0.3775101704380578
0.5428829085731715
0.5

...

0.1
0.2
0.3836591668108979
0.5169925001442313
0.7006516669551291
0.7006516669551291
0.7006516669551291
0.7006516669551291
0.7006516669551291
0.0
0.0
0.0878026757035583
0.08815291008725375
0.17386719580153945
0.1914826720606972
0.2567171959389025
0.2601423433144912
0.3889135961900567
0.5432875531581365
0.5749560055047481
0.6440245635114668
0.7035483730352763
0.7069735204108649
0.814116377553722
0.8373827947705805
0.8492099772759223
0.8955919576913638
0.9592391799822675
1.0266008076065651
1.0860470377309546
1.1733486250325418
1.276523228207145
1.4407157604665994
1.468415958844957
1.5239715144005126
1.5319080223370205
0.17483832078354541
0.22746989973091383
0.33273305762565064
0.3752254219444719
0.4146991061549982
0.5420635799151199
0.5815372641256462
0.6210109483361725
0.7175879220583387
0.757061606268865
0.7702195010057071
0.8096931852162335
0.8582157165260946
0.9318999270524104
0.9634788744208315
0.9845315059997788
1.03305403730964
1.1121728970346738
1.199892195280288
1.327302520258376

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.16666666666666666
0.3333333333333333
0.6949875002403854
0.6949875002403854
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09523809523809523
0.19047619047619047
0.38095238095238093
nan
0.0
0.0
0.0625
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
0.125
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.06410256410256411
0.07692307692307693
0.08974358974358974
0.10256410256410256
0.11538461538461538
0.1282051282051282
0.1282051282051282
0.1282051282051282
0.1282051282051282
0.1282051282051282
0.16666666666666666
0.20512820512820512
0.3205128205128205
0.3589743589743589
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
0.0606060606060606
0.1212121212121212
0.29560265158071114
0.3319662879443475
0.3865117424898021
0.44105719703525664
0.5975961147522896
0.6521415692977441
0.69759

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.037037037037037035
0.07407407407407407
0.21314398156160994
0.2686995371171655
0.324255092672721
0.36869953711716547
0.4131439815616099
0.45758842600605437
0.6353662037838321
0.6798106482282765
0.6798106482282765
0.6798106482282765
0.6798106482282765
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.1
0.2
0.25
0.3
0.3584962500721156
0.3584962500721156
0.4584962500721156
0.5584962500721156
0.6169925001442311
0.6169925001442311
0.6169925001442311
0.6169925001442311
0.6169925001442311
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.17880416676968897
0.22642321438873658
0.2740422620077842
0.2740422620077842
0.2740422620077842
0.2740422620077842
0.2740422620077842
0.2740422620077842
0.2740422620077842

0.0
0.0
0.0
0.0
nan
nan
nan
nan
0.0
0.1142857142857143
0.6549946429601652
0.7692803572458795
0.8835660715315938
0.9978517858173082
1.1049946429601654
1.2121375001030226
1.5578410717376385
1.6649839288804957
1.6649839288804957
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.25
0.5
0.75
1.646240625180289
1.646240625180289
1.646240625180289
1.646240625180289
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
0.08496250072115608
0.08496250072115608
0.08496250072115608
0.08496250072115608
0.08496250072115608
0.04248125036057804
0.04248125036057804
0.04248125036057804
0.04248125036057804
0.04248125036057804
0.04248125036057804
0.04248125036057804
0.04248125036057804
nan
nan
nan
nan
nan
nan
0.0625
0.125
0.3527410118609203
0.4152410118609203
0.7539199641905825
0.8476699641905825
0.9

,miRNA,absF
0,hsa-miR-378c,3.006674
0,hsa-mir-1321,4.416225
0,hsa-mir-378,4.132357
0,hsa-mir-26a-1,6.839010
0,hsa-mir-320a,3.046008
...,...,...
0,hsa-mir-133b,NaN
0,hsa-mir-4457,NaN
0,hsa-miR-4741,NaN
0,hsa-mir-3610,NaN


In [137]:
rTreg_M_df['RPM (reads per million)'] = rTreg_M_df['RPM (reads per million)'].astype(float)

mean_rTreg_M = rTreg_M_df.groupby('miRNA')['RPM (reads per million)'].mean()
mean_rTreg_M

miRNA
hsa-let-7a      5282.125000
hsa-let-7a*        1.000000
hsa-let-7a-1      84.428571
hsa-let-7a-2      88.142857
hsa-let-7a-3      79.714286
                   ...     
hsa-mir-942        1.000000
hsa-mir-95         1.000000
hsa-mir-98         1.000000
hsa-mir-99a        1.000000
hsa-mir-99b        1.333333
Name: RPM (reads per million), Length: 792, dtype: float64

In [138]:
aTreg_M_df['RPM (reads per million)'] = aTreg_M_df['RPM (reads per million)'].astype(float)

mean_aTreg_M = aTreg_M_df.groupby('miRNA')['RPM (reads per million)'].mean()
mean_aTreg_M

miRNA
hsa-let-7a      944.000000
hsa-let-7a*       1.000000
hsa-let-7a-1      6.111111
hsa-let-7a-2      6.125000
hsa-let-7a-3      4.125000
                   ...    
hsa-mir-942       1.000000
hsa-mir-944       1.000000
hsa-mir-98        1.000000
hsa-mir-99a       1.000000
hsa-mir-99b       1.000000
Name: RPM (reads per million), Length: 891, dtype: float64

In [139]:
df = np.log2(mean_aTreg_M / mean_rTreg_M).dropna()
df = df[np.abs(df) > 2]
df

miRNA
hsa-let-7a     -2.484260
hsa-let-7a-1   -3.788225
hsa-let-7a-2   -3.847062
hsa-let-7a-3   -4.272372
hsa-let-7b     -2.416667
                  ...   
hsa-mir-532    -2.087463
hsa-mir-574     2.222392
hsa-mir-7-1    -2.432959
hsa-mir-7-2    -2.432959
hsa-mir-7-3    -2.432959
Name: RPM (reads per million), Length: 84, dtype: float64

In [ ]:
def absF()

In [ ]:
# For heatmap 3a, we are only displaying miRNA with log2FC > 2
# r/a

for rTreg in rTreg_M_dfs:
    

In [58]:
aTreg_df

,miRNA,Chromosome,Position,Strand,Total miRNA reads,RPM (reads per million)
0,hsa-miR-378c,chr10,132760851,-,5,1
1,hsa-miR-2355-3p,chr2,207974711,-,1,1
2,hsa-mir-1321,chrX,85090785,+,8,2
3,hsa-miR-509-3p,chrX,146342050,-,1,1
4,hsa-miR-2110,chr10,115933864,-,5,1
...,...,...,...,...,...,...
348,hsa-miR-3174,chr15,90549987,+,5,1
349,hsa-miR-374a*,chrX,73507121,-,2,1
350,hsa-miR-107,chr10,91352504,-,440,29
351,hsa-miR-101,chr9,4850297,+,63,4
